<a href="https://colab.research.google.com/github/Leonatr/Programming/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_0_8_0_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Леонов Илья Сергеевич

# Задание 1

Задача: Создать чат бота для получения информации об исследованиях космоса

Описание: Создайте комплексное приложение командной строки, которое будет использоваться в качестве панели управления исследованиями космоса. Данное приложение будет обращаться к https://api.nasa.gov/ для предоставления пользователям набора информации о космосе, включая:

- Астрономическая картинка дня (APOD): Отображение APOD с пояснениями к нему.
- Фотографии с марсохода: позволяет пользователям выбирать и фильтровать фотографии с марсохода по дате и типу камеры.
- Объекты, сближающиеся с Землей (ОСЗ): Поиск и отображение информации об объекте, сближающихся с Землей, на определенную дату, включая их размеры и потенциальную опасность.
- Данные о космической погоде: Отображают последние данные о космической погоде, включая солнечные вспышки и геомагнитные бури.
Приложение должно позволять пользователям ориентироваться в этих функциях, корректно обрабатывать ошибки и обеспечивать удобство работы.

Требования:
- Пользовательский ввод: Приложение должно предложить пользователю ввести данные, чтобы выбрать, какую функцию он хочет изучить.
- Проверка данных: Убедитесь, что пользовательские данные (например, даты) проверены.
- Обработка ошибок: Корректно обрабатывайте ошибки API и неверные ответы.
- Представление данных: Представляйте данные в четкой и организованной форме.
- Опция выхода: позволяет пользователям выходить из приложения в любое время.

In [ ]:
import requests
from PIL import Image
from IPython.display import display
import io

def apod():
    url = 'https://api.nasa.gov/planetary/apod?api_key=zQnShPfUcyUv5UayIXwhcuanqpa3FhH2cqDaQkdz'
    response = requests.get(url)
    status_code = response.status_code

    if status_code in [200, 201]:
        print(response.json()["hdurl"])


        for k,v in response.json().items():
            if k in ['date', 'explanation']:
                print(v)
    else:
        print(None)


def rovers_photos():
    all_id = []
    all_name = []
    count = 0
    num=-1
    earth_date = input('Введите дату фотографии (год-месяц-день): ')

    url = f'https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos?earth_date={earth_date}&api_key=zQnShPfUcyUv5UayIXwhcuanqpa3FhH2cqDaQkdz'
    response = requests.get(url)
    status_code = response.status_code

    for i in response.json()['photos']:
        num+=1
        if response.json()['photos'][num]['camera']['name'] not in all_name:
            all_name.append(response.json()['photos'][num]['camera']['name'])
    print('Камеры, на которые в этот день были сделаны фото:', *all_name)
    camera_type = input('Введите тип камеры: ').upper()

    if status_code in [200, 201]:
        for value in response.json().values():
            for listic in value:
                if listic['camera']['id'] not in all_id:
                    if listic['camera']['name'] == camera_type:
                        count+=1
                        print(listic['img_src'])


                    all_id.append(listic['camera']['id'])

    if count == 0:
        print('Данного типа камеры не существует')


def neows():
    start_date = input('Введите дату начала поиска астероидов (год-месяц-день)')
    end_date = input('Введите дату окончания поиска астероидов (год-месяц-день)')
    count = -1

    url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=zQnShPfUcyUv5UayIXwhcuanqpa3FhH2cqDaQkdz'
    response = requests.get(url)
    status_code = response.status_code

    if status_code in [200, 201]:
        for i in response.json()['near_earth_objects'][end_date]:
            count +=1

            print('Астеройд', response.json()['near_earth_objects'][end_date][count]['name'])
            print(round((response.json()['near_earth_objects'][end_date][count]['estimated_diameter']['kilometers']['estimated_diameter_min'] + response.json()['near_earth_objects'][end_date][count]['estimated_diameter']['kilometers']['estimated_diameter_max'])/2,2), 'км')

            if response.json()['near_earth_objects'][end_date][count]['is_potentially_hazardous_asteroid'] == True:
                print('Представляет потенциальную опасность\n')
            else:
                print('Не представляет потенциальной опасности\n')


def donki():
    url1 = f'https://api.nasa.gov/DONKI/FLR?startDate=yyyy-MM-dd&endDate=yyyy-MM-dd&api_key=zQnShPfUcyUv5UayIXwhcuanqpa3FhH2cqDaQkdz'
    url2 = f'https://api.nasa.gov/DONKI/GST?startDate=yyyy-MM-dd&endDate=yyyy-MM-dd&api_key=zQnShPfUcyUv5UayIXwhcuanqpa3FhH2cqDaQkdz'
    count = -1

    response1 = requests.get(url1)
    response2 = requests.get(url2)
    status_code1 = response1.status_code
    status_code2 = response2.status_code


    if status_code1 in [200, 201] and status_code2 in [200, 201]:
        for i in response1.json():
            count+=1
            print(f'Solar flame number {count+1}:')
            print('Start Time:', response1.json()[count]['beginTime'])
            print('Peak Time:', response1.json()[count]['peakTime'])
            print('End Time:', response1.json()[count]['endTime'])
            print('Intensity:', response1.json()[count]['classType'])
            print('Source region:', response1.json()[count]['sourceLocation'])
            print('Activity ID:', response1.json()[count]['flrID'])
            print('Note:', response1.json()[count]['note'],'\n')





        print(response2.json())


chat_bot = input('''
Что вы хотите узнать?
1. Астрономическая картинка дня
2. Фотографии с марсохода
3. Объекты, сближающиеся с Землей
4. Данные о космической погоде
5. Выйти из системы
''')

while chat_bot:
    if chat_bot == '1':
        apod()
    elif chat_bot == '2':
        rovers_photos()
    elif chat_bot == '3':
        neows()
    elif chat_bot == '4':
        donki()
    elif chat_bot == '5':
        break
    chat_bot = input('''
Что вы хотите узнать?
1. Астрономическая картинка дня
2. Фотографии с марсохода
3. Объекты, сближающиеся с Землей
4. Данные о космической погоде
5. Выйти из системы
''')




Что вы хотите узнать?
1. Астрономическая картинка дня
2. Фотографии с марсохода
3. Объекты, сближающиеся с Землей
4. Данные о космической погоде
5. Выйти из системы
2
Введите дату фотографии (год-месяц-день): 2022-11-11
Камеры, на которые в этот день были сделаны фото: CHEMCAM NAVCAM
Введите тип камеры: NAVCAM
https://mars.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/03649/opgs/edr/ncam/NLB_721422382EDR_F0980908CCAM04648M_.JPG

Что вы хотите узнать?
1. Астрономическая картинка дня
2. Фотографии с марсохода
3. Объекты, сближающиеся с Землей
4. Данные о космической погоде
5. Выйти из системы
3
Введите дату начала поиска астероидов (год-месяц-день)2022-11-11
Введите дату окончания поиска астероидов (год-месяц-день)2022-11

Что вы хотите узнать?
1. Астрономическая картинка дня
2. Фотографии с марсохода
3. Объекты, сближающиеся с Землей
4. Данные о космической погоде
5. Выйти из системы
3
Введите дату начала поиска астероидов (год-месяц-день)2022-11-11
Введите дату окончания пои

# Задание 2

Описание задачи

Цель этой задачи - создать скрипт на Python, который взаимодействует с API Чикагского института искусств (https://api.artic.edu/docs/) для извлечения и отображения произведений искусства. Скрипт должен позволять пользователям просматривать работы по страницам, фильтровать их по имени художника и просматривать подробную информацию о выбранных произведениях искусства. Ниже приведены требования и функциональные возможности, которые необходимо реализовать:

Требования:
Извлекать произведения искусства:

- Создайте функцию, которая извлекает список произведений искусства из API Чикагского института искусств.
Функция должна принимать параметр page для разбивки на страницы и возвращать список произведений искусства вместе с информацией о разбивке на страницы.
Фильтровать произведения искусства:

- Реализуйте функцию, которая фильтрует список произведений искусства на основе имени указанного художника. Функция должна возвращать список работ, которые соответствуют имени художника (без учета регистра).
Отображать подробную информацию об оформлении:

- Напишите функцию, которая отображает названия работ для пользователя и позволяет ему выбрать одну из них, введя соответствующий номер.
После выбора функция должна отображать подробную информацию о выбранном произведении, включая название, исполнителя, дату и носитель.
Разбивка на страницы и взаимодействие с пользователем:

- Создайте основную функцию, которая управляет выборкой произведений и взаимодействием с пользователем.

Разрешите пользователям перемещаться по страницам с произведениями искусства, выполнять фильтрацию по исполнителю или выходить из программы.

Если страниц с произведениями искусства несколько, укажите варианты перехода к следующей странице, предыдущей странице, фильтрации по исполнителю или выхода из программы.

In [ ]:
import requests

def artworks(url):
    all_picture = []
    response = requests.get(url)
    status_code = response.status_code
    count = -1
    num = 0

    if status_code in [200, 201]:
        for i in response.json()['data']:
            count += 1
            num+=1
            all_picture.append(num)
            all_picture.append(response.json()['data'][count]['title'])
        return all_picture,response.json()['data']
    return None


def artists_name(url):
    response = requests.get(url)
    status_code = response.status_code

    all_artists = []
    count = -1
    count1 = -1
    num = 0

    if status_code in [200,201]:
        print('Все художники с этой страницы:')

        for i in response.json()['data']:
            count1 += 1
            artists_title = response.json()['data'][count1]['artist_title']
            if artists_title not in all_artists:
                all_artists.append(artists_title)
                print(artists_title)

        artist_name = input('Введите имя художника: ')

        for i in response.json()['data']:
            count += 1
            if artist_name == response.json()['data'][count]['artist_title']:
                num+=1
                print('Картина номер', str(num)+ ':',response.json()['data'][count]['title'])


def choosing_painting(url):
    all_picture = artworks(url)[0]
    url = artworks(url)[1]
    print('Все картины:')
    for i in range(0,len(all_picture),2):
        print(all_picture[i], all_picture[i+1])
    num_pic = int(input('\nВыберите номер интересующей вас картины: '))
    print('Название:', all_picture[num_pic*2-1])
    print('Автор:', url[num_pic]['artist_title'])
    print('Дата окончания работы:', url[num_pic]['date_end'])
    print('Страна, где создана:', url[num_pic]['place_of_origin'])


def main():
    print('Здравствуйте, здесь вы можете изучить библиотеку Чикагского института искусств!')
    number_page = input('Введите номер страницы (от 1 до 1264): ')




    while number_page:
        url = f'https://api.artic.edu/api/v1/artworks?page={number_page}&limit=100'

        choose = input('''Вы можете выбрать какую-то конкретную картину с этой страницы (напишите 1), или отфильтровать страницу по автору (напишите 2): ''')
        if choose == '1':
            choosing_painting(url)
        if choose == '2':
            artists_name(url)

        number_page = input('\n Если хотите продолжить поиск введите номер страницы (от 1 до 1264). Если хотите закончить. нажмите Enter: ')





main()


Здравствуйте, здесь вы можете изучить библиотеку Чикагского института искусств!
Введите номер страницы (от 1 до 1264): 23
Вы можете выбрать какую-то конкретную картину с этой страницы (напишите 1), или отфильтровать страницу по автору (напишите 2): 2
Все художники с этой страницы:
Venetian
British
British School
Netherlandish
Italian
Roman
Salomon van Ruysdael
Théodore Rousseau
Vincent van Gogh
Harald Oscar Sohlberg
Gordon Parks
Johann Heinrich Stobwasser
Anton Raffael Mengs
Willem de Kooning
Amanda Williams
Max Ernst
Paula Modersohn-Becker
Navajo (Diné)
Giulio Romano
North Netherlandish
Master of the Lille Adoration
Quentin Massys
Follower of François Clouet
Frans Floris, I
Flemish
None
Mimura Chikuho
Circle of Adriaen Isenbrant
Workshop of Adriaen Isenbrant
Daniel Bretschneider, the younger
Johann Koerbecke
Jacob Jordaens
Archibald Knox
Abraham Janssens
Heinrich Jansen
Jan Lievens
Hans Maler
Johann Carl Loth
Dirck van der Lisse
Boetius Adams Bolswert
Anthony van Dyck
Jean Mignon
Pier

# Задание 3

Задача: Создать программу по управлению портфелем криптовалют

Цель: Создать скрипт на Python, который извлекает цены на криптовалюты в режиме реального времени, позволяет пользователям управлять портфелем криптовалют, вычисляет общую стоимость портфеля, отслеживает изменения цен и предоставляет исторические данные о ценах для анализа.

Требования:
Получение текущих цен на криптовалюты:

Используйте https://docs.coingecko.com/ для получения актуальных цен на список криптовалют.

Управление портфелем:

- Позволяет пользователю создавать портфель криптовалют и управлять им, указывая количество каждой криптовалюты, которой он владеет.
- Расчитывает общую стоимость портфеля в указанной фиатной валюте (например, долларах США).

Отслеживание изменения цен:

- Отображение процентного изменения цены для каждой криптовалюты в портфеле за последние 24 часа.
- Выделите все криптовалюты, стоимость которых значительно увеличилась или снизилась.

Поиск исторических данных о ценах:

- Получение исторических данных о ценах на указанную криптовалюту за последнюю неделю.
- Предоставьте пользователю возможность визуализировать эти данные в простом текстовом формате (например, цены за день).

Взаимодействие с пользователем:

- Реализуйте интерфейс командной строки для ввода данных пользователем.
- Предоставьте опции для получения текущих цен, управления портфелем, просмотра изменений цен или анализа исторических данных.


# Дополнительно: Задание 4

Задание 4: Проектное

Вам необходимо самостоятельно найти откртое API предоставляющее информацию в открытом доступе и реализовать собственный проект!


Критерии приемки результата:

- Проект включает в себя не менее 5 возможостей для пользователя
- Проект позволяет использовать все возможности проекта пользователю при помощи взаимодействия через коммандную строку
- Проект работает с открытым API (это значит что при проверке вашей работы преподавателем, преподавателю необходимо просто запустить ячейку с кодом вашего проекта и она будет работать без дополнительных манипуляции)
- Проект должен обязательно включать в себя ряд используемых конструкции:
    - Функции
    - Условные конструкции
    - Ввод/вывод
    - Словари/Списки
- Допускается использование библиотек:
    - requests
    - datetime
    - random

**Здесь добавьте описание вашего проекта**

In [ ]:
#  А здесь код